In [15]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense
from tensorflow.keras.regularizers import l1
from tqdm import tqdm

In [16]:
EPOCHS = 100

In [17]:
# List all available devices detected by TensorFlow
print("Available devices:")
devices = tf.config.list_physical_devices()
for device in devices:
    print(device)

Available devices:
PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU')


## Load and prepare data

In [18]:
# Load the data
data = pd.read_csv("../dataset.csv")

In [19]:
# Separate features and target
X = data.drop("Label", axis=1)
y = data["Label"]

In [20]:
# Convert categorical data to numeric if necessary
X = pd.get_dummies(X, sparse=True)

In [21]:
# Split the data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

In [22]:
# Standardize the data
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

/Users/odai/cs5525-research-paper-submission/.venv/lib/python3.11/site-packages/sklearn/utils/validation.py:869: UserWarning: pandas.DataFrame with sparse columns found.It will be converted to a dense numpy array.
  warnings.warn(
/Users/odai/cs5525-research-paper-submission/.venv/lib/python3.11/site-packages/sklearn/utils/validation.py:869: UserWarning: pandas.DataFrame with sparse columns found.It will be converted to a dense numpy array.
  warnings.warn(
/Users/odai/cs5525-research-paper-submission/.venv/lib/python3.11/site-packages/sklearn/utils/validation.py:1059: FutureWarning: Allowing arbitrary scalar fill_value in SparseDtype is deprecated. In a future version, the fill_value must be a valid value for the SparseDtype.subtype.
  if np.may_share_memory(array, array_orig):
/Users/odai/cs5525-research-paper-submission/.venv/lib/python3.11/site-packages/sklearn/utils/validation.py:869: UserWarning: pandas.DataFrame with sparse columns found.It will be converted to a dense numpy arr

## Build an AutoEncoder

In [23]:
# Define the encoder with sparsity regularization
input_layer = Input(shape=(X_train_scaled.shape[1],))
encoded = Dense(128, activation="relu")(input_layer)

# Add L1 activity regularizer to the next dense layer
encoded = Dense(64, activation="relu", activity_regularizer=l1(10e-5))(encoded)

In [24]:
# Define the decoder
decoded = Dense(128, activation="relu")(encoded)
decoded = Dense(X_train_scaled.shape[1], activation="sigmoid")(decoded)

In [25]:
# Define the autoencoder
autoencoder = Model(input_layer, decoded)

# Encoder for predictions
encoder = Model(input_layer, encoded)

# Prediction model
predictor = Dense(1, activation="sigmoid")(encoded)
prediction_model = Model(input_layer, predictor)

# Compile models
autoencoder.compile(optimizer="adam", loss="mse")
prediction_model.compile(
    optimizer="adam", loss="binary_crossentropy", metrics=["accuracy"]
)

In [26]:
# Display the model summary
autoencoder.summary()
prediction_model.summary()

Model: "functional_7"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_1 (InputLayer)      │ (None, 20090)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (None, 128)            │     2,571,648 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_6 (Dense)                 │ (None, 64)             │         8,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_7 (Dense)                 │ (None, 128)            │         8,320 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_8 (Dense)                 │ (None, 20090)          │     2,591,610 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 5,179,834 (19.76 MB)

 Trainable params: 5,179,834 (19.76 MB)

 Non-trainable params: 0 (0.00 B)

Model: "functional_11"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_1 (InputLayer)      │ (None, 20090)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (None, 128)            │     2,571,648 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_6 (Dense)                 │ (None, 64)             │         8,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_9 (Dense)                 │ (None, 1)              │            65 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,579,969 (9.84 MB)

 Trainable params: 2,579,969 (9.84 MB)

 Non-trainable params: 0 (0.00 B)

## Train and evaluate

In [27]:
# Train the autoencoder
autoencoder.fit(
    X_train_scaled, X_train_scaled, epochs=EPOCHS, batch_size=256, validation_split=0.2
)

Epoch 1/100
25/25 ━━━━━━━━━━━━━━━━━━━━ 3s 81ms/step - loss: 1.4154 - val_loss: 1.1007
Epoch 2/100
25/25 ━━━━━━━━━━━━━━━━━━━━ 1s 54ms/step - loss: 1.0296 - val_loss: 1.0041
Epoch 3/100
25/25 ━━━━━━━━━━━━━━━━━━━━ 1s 55ms/step - loss: 0.9733 - val_loss: 0.9528
Epoch 4/100
25/25 ━━━━━━━━━━━━━━━━━━━━ 1s 49ms/step - loss: 0.9177 - val_loss: 0.9271
Epoch 5/100
25/25 ━━━━━━━━━━━━━━━━━━━━ 1s 53ms/step - loss: 0.8774 - val_loss: 0.9129
Epoch 6/100
25/25 ━━━━━━━━━━━━━━━━━━━━ 2s 60ms/step - loss: 0.8528 - val_loss: 0.9037
Epoch 7/100
25/25 ━━━━━━━━━━━━━━━━━━━━ 1s 49ms/step - loss: 0.8327 - val_loss: 0.8996
Epoch 8/100
25/25 ━━━━━━━━━━━━━━━━━━━━ 1s 56ms/step - loss: 0.8174 - val_loss: 0.8982
Epoch 9/100
25/25 ━━━━━━━━━━━━━━━━━━━━ 1s 56ms/step - loss: 0.8101 - val_loss: 0.8975
Epoch 10/100
25/25 ━━━━━━━━━━━━━━━━━━━━ 1s 57ms/step - loss: 0.8068 - val_loss: 0.8971
Epoch 11/100
25/25 ━━━━━━━━━━━━━━━━━━━━ 1s 50ms/step - loss: 0.8050 - val_loss: 0.8968
Epoch 12/100
25/25 ━━━━━━━━━━━━━━━━━━━━ 2s 64ms/step

In [28]:
# Train the prediction model
prediction_model.fit(
    X_train_scaled, y_train, epochs=EPOCHS, batch_size=256, validation_split=0.2
)

Epoch 1/100
25/25 ━━━━━━━━━━━━━━━━━━━━ 1s 36ms/step - accuracy: 0.4906 - loss: 0.6932 - val_accuracy: 0.4975 - val_loss: 0.8027
Epoch 2/100
25/25 ━━━━━━━━━━━━━━━━━━━━ 1s 30ms/step - accuracy: 0.5003 - loss: 0.6932 - val_accuracy: 0.4981 - val_loss: 0.8028
Epoch 3/100
25/25 ━━━━━━━━━━━━━━━━━━━━ 1s 36ms/step - accuracy: 0.4923 - loss: 0.6932 - val_accuracy: 0.5006 - val_loss: 0.8030
Epoch 4/100
25/25 ━━━━━━━━━━━━━━━━━━━━ 1s 40ms/step - accuracy: 0.5068 - loss: 0.6931 - val_accuracy: 0.4994 - val_loss: 0.8030
Epoch 5/100
25/25 ━━━━━━━━━━━━━━━━━━━━ 1s 43ms/step - accuracy: 0.5015 - loss: 0.6931 - val_accuracy: 0.4981 - val_loss: 0.8028
Epoch 6/100
25/25 ━━━━━━━━━━━━━━━━━━━━ 1s 51ms/step - accuracy: 0.5070 - loss: 0.6931 - val_accuracy: 0.4988 - val_loss: 0.8029
Epoch 7/100
25/25 ━━━━━━━━━━━━━━━━━━━━ 1s 43ms/step - accuracy: 0.4951 - loss: 0.6932 - val_accuracy: 0.5006 - val_loss: 0.8030
Epoch 8/100
25/25 ━━━━━━━━━━━━━━━━━━━━ 1s 40ms/step - accuracy: 0.4951 - loss: 0.6932 - val_accuracy: 0.

In [29]:
# Evaluate the prediction model
test_loss, test_acc = prediction_model.evaluate(X_test_scaled, y_test)
print("Test Accuracy: {:.2f}%".format(test_acc * 100))

63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.5191 - loss: 0.6931
Test Accuracy: 50.50%


In [30]:
# Pickle the models
autoencoder.save("checkpoints/sae.h5")
prediction_model.save("checkpoints/sae_prediction.h5")